In [49]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import time
import sys


In [50]:
csv_path = "Group6_Project4_data.csv"
bb_players_df = pd.read_csv(csv_path)
bb_players_df.head(10)

,player_name,team,conf,GP,Min_per,Ortg,usg,eFG,TS_per,ORB_per,...,Tweet Data,Trending Data,Height,Wingspan,Weight,Position,Draft Year,Draft Pick Y/N,NBA Rings,NBA Champion YN
0,DeAndrae Ross,South Alabama,SB,26,29.5,97.3,16.6,42.5,44.43,1.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
1,Pooh Williams,Utah St.,WAC,34,60.9,108.3,14.9,52.4,54.48,3.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
2,Jesus Verdejo,South Florida,BE,27,72.0,96.2,21.8,45.7,47.98,2.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
3,Mike Hornbuckle,Pepperdine,WCC,30,44.5,97.7,16.0,53.6,53.69,4.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
4,Anthony Brown,Pacific,BW,33,56.2,96.5,22.0,52.8,54.31,8.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
5,Nick Rodgers,Butler,Horz,6,0.7,0.0,0.0,0.0,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
6,Dana Smith,Longwood,ind,27,77.8,104.8,23.0,53.4,56.30,6.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
7,Matt Beck,Fordham,A10,19,10.4,131.9,3.3,90.0,90.00,2.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
8,Justin Drummond,Wagner,NEC,30,82.8,99.7,20.5,48.8,53.07,2.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
9,Jamal Smith,Wagner,NEC,30,80.4,92.5,23.0,43.5,45.29,6.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


In [51]:
year=[]
team=[]
player_name = []
player_height = []
player_weight = []
player_position = []
process_status = []
start_row=1000
end_row=2000
for index, row in bb_players_df.iterrows():
    height=0
    weight=0
    position="NA"
    print(index,"-" ,row['player_name'])
    if index < start_row:
        continue
    if index > end_row:
        break
    player = row['player_name'].lower().replace(".","").replace("'","").replace(",","").replace("`","")
    first_last = player.split()
    sports_ref_player =""
    for windex,word in enumerate(first_last):
        if windex==0 and len(first_last) > 1:
            sports_ref_player = word
        elif windex==0 and len(first_last) == 1:
            sports_ref_player = word + "-1"
        elif windex > 0 and len(first_last) > windex +1:
            sports_ref_player = sports_ref_player + "-" +   word
        elif windex > 0 and len(first_last) == windex+1:
            sports_ref_player = sports_ref_player + "-" +   word + "-1"
            
    print(sports_ref_player)
#    sports_ref_player = first_last[0] + "-" + first_last[1] + "-1"
    milliseconds = 100
    seconds = 0.001 * milliseconds
    start_time = time.time()
    time.sleep(seconds)
    stop_time = time.time()
#    time.sleep(1)
    player_name.append(row['player_name'])
    year.append(row['year'])
    team.append(row['team'])
    url = 'https://www.sports-reference.com/cbb/players/' + sports_ref_player+'.html'
    try:
        response = requests.get(url)
        if response.text.find("Position:") > -1 and response.text.find('itemprop="height"') > -1 :
            print("Found url")
        else :
            response = requests.get(url.replace("-1","-2"))
            
        soup = BeautifulSoup(response.text, 'html.parser')
        if response.text.find("Position:") > -1 and response.text.find('itemprop="height"') > -1 : 
            results = soup.find_all('div', id='info')
            #print(results[0])
            ptags = results[0].find_all('p')
            print(ptags[0].contents[2].strip())
            position=ptags[0].contents[2].strip()
            player_position.append(position)
            print(ptags[0].contents[3].contents[-1].strip())
            poop = ptags[0].contents[3].contents[-1].strip().replace("(","").replace(")","").replace("cm","").replace("kg","")
            print(poop)
            height_weight= poop.split(",\xa0")
            if len(height_weight) > 1 :
                height=height_weight[0]
                weight=height_weight[1]
            elif len(height_weight) > 0 :
                height=height_weight[0]
            
#            height,weight = poop.split(",\xa0")
            print(height,"-" ,weight)
            player_height.append(height)
            player_weight.append(weight)
            process_status.append("Success")
        else:
            print("Not Found in :",response.text)
            process_status.append("Failure: Did not find player")
            player_position.append(-1)
            player_height.append(-1)
            player_weight.append(-1)
    except:
            (type, value, tb) = sys.exc_info()
            err = "%s" % value
            process_status.append("Failure: Other Error")
            player_position.append(-100)
            player_height.append(-100)
            player_weight.append(-100)
    

0 - DeAndrae Ross
1 - Pooh Williams
2 - Jesus Verdejo
3 - Mike Hornbuckle
4 - Anthony Brown
5 - Nick Rodgers
6 - Dana Smith
7 - Matt Beck
8 - Justin Drummond
9 - Jamal Smith
10 - Tyrell Biggs
11 - Alex Hornat
12 - Simon Harris
13 - Gary Wilkinson
14 - Tawrence Walton
15 - Garrett Temple
16 - Carlos Strong
17 - Sherrod Smith
18 - Eulis Stephens
19 - Scott Brittain
20 - Brett Gifford
21 - Valdas Sirutis
22 - Corey Lowe
23 - Grant Leiendecker
24 - Patrick Bouli
25 - Antoine Pearson
26 - Jamel Ferguson
27 - Chris Harris
28 - T.J. Topercer
29 - Brandon Adams
30 - Darryl Crawford
31 - Carlin Hughes
32 - Jabari Deshields
33 - Charles Jenkins
34 - Pierre Curtis
35 - Greg Washington
36 - Matt Parker
37 - Shane Laurie
38 - Alonzo Gee
39 - Brandon Hollinger
40 - Tyrese Rice
41 - Evan Neisler
42 - Mike Wilson
43 - Moussa Camara
44 - Otis Polk
45 - Chris Knight
46 - Marc Larson
47 - Landon Milbourne
48 - Eric Hayes
49 - Jerome Burney
50 - Levi Knutson
51 - Jeremy Wise
52 - Frank Hassell
53 - Josh A

632 - Joey Shank
633 - Kenny Thomas
634 - Alex Woodhouse
635 - Eric Hall
636 - Devonte Thomas
637 - Ade Dagunduro
638 - Ricky Bieszki
639 - Derick Nelson
640 - Louis Brookins
641 - James Hennessy
642 - Jason Duty
643 - Erik Kangas
644 - Rodney Stokes
645 - Mike Walker
646 - Jack McClinton
647 - Jacob Trader
648 - Mamadi Diane
649 - Courtney Fells
650 - Ben McCauley
651 - Brandon Costner
652 - Brandon Johnson
653 - Chris Lewis
654 - Deion James
655 - Trevor Morris
656 - Tim Frye
657 - Kevin Swecker
658 - Brandon Patterson
659 - Ethan Niedermeyer
660 - Kyle Brucculeri
661 - Taishi Ito
662 - Yusef Smith
663 - Michael Boone
664 - Bryan Mullins
665 - Ramon Harris
666 - Jeremi Booth
667 - Coy Custer
668 - Travis Gabbidon
669 - Luka Drca
670 - Devan Carter
671 - Nick Stallings
672 - Jeff Smith
673 - Brandon Giles
674 - Jeremy Saffore
675 - Blake Thompson
676 - Brigham Waginger
677 - Lazar Hayward
678 - David Cubillan
679 - Mikhail Torrance
680 - Paul Harris
681 - Dan Mavraides
682 - Ryan Derm

Found url
Forward
(196cm, 86kg)
196, 86
196 - 86
1012 - Eniel Polynice
eniel-polynice-1
Found url
Guard
(196cm, 99kg)
196, 99
196 - 99
1013 - Adam May
adam-may-1
Found url
Forward
(203cm, 92kg)
203, 92
203 - 92
1014 - Andrew Keister
andrew-keister-1
Found url
Center
(206cm, 106kg)
206, 106
206 - 106
1015 - Eric Zastoupil
eric-zastoupil-1
Found url
Forward
(201cm, 94kg)
201, 94
201 - 94
1016 - Chris Walker
chris-walker-1
Found url
Forward
(206cm, 105kg)
206, 105
206 - 105
1017 - Tyrell Thompson
tyrell-thompson-1
Found url
Guard
(190cm, 89kg)
190, 89
190 - 89
1018 - Jon Sizemore
jon-sizemore-1
Found url
Guard
(193cm, 89kg)
193, 89
193 - 89
1019 - Jerome Tillman
jerome-tillman-1
Found url
Forward
(198cm, 106kg)
198, 106
198 - 106
1020 - Eric Meister
eric-meister-1
Found url
Center
(203cm, 102kg)
203, 102
203 - 102
1021 - Emmanuel Jones
emmanuel-jones-1
Found url
Forward
(201cm, 97kg)
201, 97
201 - 97
1022 - Nick Weiermiller
nick-weiermiller-1
Found url
Guard
(185cm, 92kg)
185, 92
185 - 92

Guard
(188cm, 79kg)
188, 79
188 - 79
1108 - Jon Scheyer
jon-scheyer-1
Found url
Guard
(196cm, 86kg)
196, 86
196 - 86
1109 - Jim Griffin
jim-griffin-1
Found url
Guard
(201cm, 92kg)
201, 92
201 - 92
1110 - Grayson Flittner
grayson-flittner-1
Found url
Guard
(183cm, 90kg)
183, 90
183 - 90
1111 - Tom Taylor
tom-taylor-1
Found url
Center
(211cm, 99kg)
211, 99
211 - 99
1112 - Harry Marshall
harry-marshall-1
Found url
Guard
(183cm, 86kg)
183, 86
183 - 86
1113 - Chris Armstrong
chris-armstrong-1
Guard
(185cm, 88kg)
185, 88
185 - 88
1114 - Olawale Alesh
olawale-alesh-1
Not Found in : 
<!DOCTYPE html>
<html data-version="klecko-" data-root="/home/cbb/build" itemscope itemtype="https://schema.org/WebSite" lang="en" class="no-js" >
<head id="suppress_all_ads">
    <meta charset="utf-8">
    <meta http-equiv="x-ua-compatible" content="ie=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />
    <link rel="dns-prefetch" href="https://d2p3bygnnzw9w3.cl

Found url
Center
(211cm, 127kg)
211, 127
211 - 127
1116 - Wayne Chism
wayne-chism-1
Found url
Forward
(206cm, 111kg)
206, 111
206 - 111
1117 - Josh Tabb
josh-tabb-1
Found url
Guard
(193cm, 88kg)
193, 88
193 - 88
1118 - Kelvin Lewis
kelvin-lewis-1
Found url
Guard
(193cm, 88kg)
193, 88
193 - 88
1119 - Brandon Bush
brandon-bush-1
Found url
Guard
(201cm, 88kg)
201, 88
201 - 88
1120 - Keith Wilkinson
keith-wilkinson-1
Found url
Forward
(208cm, 102kg)
208, 102
208 - 102
1121 - Chris Hayes
chris-hayes-1
Found url
Guard
(193cm, 92kg)
193, 92
193 - 92
1122 - John Kast
john-kast-1
Found url
Guard
(188cm, 82kg)
188, 82
188 - 82
1123 - Michael Rogers
michael-rogers-1
Found url
Guard
(198cm, 104kg)
198, 104
198 - 104
1124 - Marshall Carrell
marshall-carrell-1
Found url
Forward
(206cm, 104kg)
206, 104
206 - 104
1125 - Stephen Duckett
stephen-duckett-1
Found url
Guard
(193cm, 86kg)
193, 86
193 - 86
1126 - Joe Hughes
joe-hughes-1
Found url
Forward
(198cm, 92kg)
198, 92
198 - 92
1127 - Herb Tanner
herb

...

Found url
Guard
(183cm, 77kg)
183, 77
183 - 77
1593 - Chad Tomko
chad-tomko-1
Found url
Guard
(180cm, 72kg)
180, 72
180 - 72
1594 - Matt Dorlack
matt-dorlack-1
Found url
Center
(211cm, 114kg)
211, 114
211 - 114
1595 - Darius James
darius-james-1
Found url
Guard
(183cm, 77kg)
183, 77
183 - 77
1596 - Solomon Tat
solomon-tat-1
Found url
Forward
(196cm, 98kg)
196, 98
196 - 98
1597 - Kit Avery
kit-avery-1
Found url
Guard
(188cm, 83kg)
188, 83
188 - 83
1598 - Chad Toppert
chad-toppert-1
Found url
Guard
(201cm, 97kg)
201, 97
201 - 97
1599 - Shane Johannsen
shane-johannsen-1
Found url
Forward
(201cm, 106kg)
201, 106
201 - 106
1600 - Christian Soto
christian-soto-1
Found url
Forward
(203cm, 99kg)
203, 99
203 - 99
1601 - Cheick Diakite
cheick-diakite-1
Found url
Forward
(203cm, 98kg)
203, 98
203 - 98
1602 - A.D. Vassallo
ad-vassallo-1
Found url
Guard
(198cm, 97kg)
198, 97
198 - 97
1603 - John Lamb
john-lamb-1
Found url
Guard
(185cm, 83kg)
185, 83
185 - 83
1604 - Spencer Laurie
spencer-laurie-1
F

Found url
Guard
(185cm, 92kg)
185, 92
185 - 92
1690 - Andrew Goudelock
andrew-goudelock-1
Found url
Guard
(188cm, 90kg)
188, 90
188 - 90
1691 - Chris Linton
chris-linton-1
Found url
Forward
(198cm, 88kg)
198, 88
198 - 88
1692 - Junior Hairston
junior-hairston-1
Found url
Forward
(203cm, 92kg)
203, 92
203 - 92
1693 - Anthony Nelson
anthony-nelson-1
Found url
Guard
(185cm, 79kg)
185, 79
185 - 79
1694 - Marquis Weddle
marquis-weddle-1
Found url
Guard
(185cm, 83kg)
185, 83
185 - 83
1695 - Benzor Simmons
benzor-simmons-1
Found url
Guard
(196cm, 97kg)
196, 97
196 - 97
1696 - Djero Riedewald
djero-riedewald-1
Found url
Forward
(203cm, 99kg)
203, 99
203 - 99
1697 - Evan Washington
evan-washington-1
Found url
Guard
(193cm, 90kg)
193, 90
193 - 90
1698 - Phillip Brown
phillip-brown-1
Found url
Center
(201cm, 97kg)
201, 97
201 - 97
1699 - Mike McLain
mike-mclain-1
Found url
Center
(203cm, 106kg)
203, 106
203 - 106
1700 - Archie Rose
archie-rose-1
Found url
Guard
(196cm, 95kg)
196, 95
196 - 95
1701

Found url
Guard
(178cm, 74kg)
178, 74
178 - 74
1788 - Ryan Kannegieter
ryan-kannegieter-1
Found url
Guard
(188cm, 77kg)
188, 77
188 - 77
1789 - Tai Wesley
tai-wesley-1
Found url
Forward
(201cm, 108kg)
201, 108
201 - 108
1790 - Melvin Crowder
melvin-crowder-1
Found url
Guard
(193cm, 97kg)
193, 97
193 - 97
1791 - Cedric Jackson
cedric-jackson-1
Found url
Guard
(188cm, 86kg)
188, 86
188 - 86
1792 - Manny Quezada
manny-quezada-1
Found url
Guard
(188cm, 83kg)
188, 83
188 - 83
1793 - Dan Waterstradt
dan-waterstradt-1
Found url
Center
(211cm, 108kg)
211, 108
211 - 108
1794 - Ed Nixon
ed-nixon-1
Found url
Guard
(193cm, 95kg)
193, 95
193 - 95
1795 - Daniel Huggins
daniel-huggins-1
Found url
Guard
(190cm, 92kg)
190, 92
190 - 92
1796 - Chris Hyche
chris-hyche-1
Found url
Guard
(196cm, 92kg)
196, 92
196 - 92
1797 - Dominique Archie
dominique-archie-1
Found url
Forward
(201cm, 90kg)
201, 90
201 - 90
1798 - Robinson Louisme
robinson-louisme-1
Found url
Center
(203cm, 120kg)
203, 120
203 - 120
1799 -

Found url
Forward
(201cm, 97kg)
201, 97
201 - 97
1885 - Gani Lawal
gani-lawal-1
Found url
Forward
(206cm, 106kg)
206, 106
206 - 106
1886 - Will Thomas
will-thomas-1
Found url
Forward
(201cm, 104kg)
201, 104
201 - 104
1887 - DaShawn Wright
dashawn-wright-1
Found url
Guard
(193cm, 90kg)
193, 90
193 - 90
1888 - Kellen Brand
kellen-brand-1
Found url
Guard
(185cm, 83kg)
185, 83
185 - 83
1889 - Stephan Martin
stephan-martin-1
Found url
Guard
(183cm, 81kg)
183, 81
183 - 81
1890 - Ryan Shields
ryan-shields-1
Found url
Forward
(203cm, 111kg)
203, 111
203 - 111
1891 - Colin McIntosh
colin-mcintosh-1
Found url
Forward
(203cm, 95kg)
203, 95
203 - 95
1892 - Jordan Dean
jordan-dean-1
Found url
Forward
(198cm, 92kg)
198, 92
198 - 92
1893 - Mike Trimboli
mike-trimboli-1
Found url
Guard
(185cm, 81kg)
185, 81
185 - 81
1894 - Derek Brooks
derek-brooks-1
Found url
Forward
(196cm, 90kg)
196, 90
196 - 90
1895 - Chris Balham
chris-balham-1
Found url
Forward
(203cm, 106kg)
203, 106
203 - 106
1896 - Jordan Lem

Found url
Guard
(188cm, 86kg)
188, 86
188 - 86
1981 - Ty Gough
ty-gough-1
Found url
Center
(208cm, 115kg)
208, 115
208 - 115
1982 - Danny Sumner
danny-sumner-1
Found url
Forward
(201cm, 102kg)
201, 102
201 - 102
1983 - David Schneider
david-schneider-1
Found url
Guard
(190cm, 86kg)
190, 86
190 - 86
1984 - Adrian Rogers
adrian-rogers-1
Found url
Forward
(201cm, 106kg)
201, 106
201 - 106
1985 - Frane Markusovic
frane-markusovic-1
Found url
Center
(208cm, 108kg)
208, 108
208 - 108
1986 - Damon Jones
damon-jones-1
Found url
Guard
(190cm, 83kg)
190, 83
190 - 83
1987 - Chris Howard
chris-howard-1
Found url
Guard
(190cm, 83kg)
190, 83
190 - 83
1988 - Trant Simpson
trant-simpson-1
Guard
(178cm, 72kg)
178, 72
178 - 72
1989 - Jimmy Harding
jimmy-harding-1
Found url
Guard
(196cm, 74kg)
196, 74
196 - 74
1990 - David Vallins
david-vallins-1
Found url
Guard
(175cm, 68kg)
175, 68
175 - 68
1991 - Barry Stewart
barry-stewart-1
Found url
Guard
(190cm, 77kg)
190, 77
190 - 77
1992 - Sean Haluska
sean-halu

In [52]:
print(len(player_name),len(player_position),len(player_height),len(process_status))

1001 1001 1001 1001


In [54]:
import numpy as np
player_add_info_dict = {"player_name":player_name,"year":year,"team":team,"player_position":player_position,"player_height":player_height,"player_weight":player_weight,"process_status":process_status}
player_add_info_df=pd.DataFrame(data=player_add_info_dict)
#process_status

#player_add_info_df["process_status"] = np.where(player_add_info_df["process_status"] == "Success", "Success", "Failure")
player_add_info_df

,player_name,year,team,player_position,player_height,player_weight,process_status
0,Paul Nelson,2009,Yale,Center,208,104,Success
1,Jordan Gibson,2009,Yale,Forward,196,88,Success
2,Sam Willard,2009,Pacific,Forward,206,102,Success
3,Robbie Myers,2009,Rider,Forward,206,115,Success
4,Kevyn Green,2009,Southeastern Louisiana,Guard,196,92,Success
...,...,...,...,...,...,...,...
996,Craig Brackins,2009,Iowa St.,Forward,208,104,Success
997,Andrew Sorenson,2009,Gonzaga,Guard,188,78,Success
998,Edwin Lashley,2009,Saint Joseph's,Guard,193,92,Success
999,Damir Suljagic,2009,Texas Tech,Forward,203,113,Success


In [55]:
player_file = "player_add_info_"+str(start_row)+"_"+str(end_row)+".csv"
print(player_file)
player_add_info_df.to_csv(player_file)

player_add_info_1000_2000.csv


# URL of page to be scraped
url = 'https://www.sports-reference.com/cbb/players/travis-hill-1.html'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')


results = soup.find_all('div', id='info')
#print(results[0])
ptags = results[0].find_all('p')
print(ptags[0].contents[2].strip())
print(ptags[0].contents[3].contents[-1].strip())
poop = ptags[0].contents[3].contents[-1].strip().replace("(","").replace(")","").replace("cm","").replace("kg","")
print(poop)
height,weight = poop.split(",\xa0")
print(height,"-" ,weight)

#height_cm,weight_kg=tuple(ptags[0].contents[3].contents[-1].strip())
#print(height_cm)

print(ptags)
print(len(ptags))
#print(ptags[0])
#new_ptags = ptags[0].find_all('p')
print(new_ptags)
#print(new_ptags[0].text)
#print(new_ptags[0].text)
#print(new_ptags[0].text)
#print(ptags[2].text)
#print(ptags[3].text)
#print(ptags[4].text)
#for ptag in ptags:
#    print(ptag.text)

import re
results = soup.find_all('div', id='meta')
#print(results[0])
strong = results[0].find('p')
split_meta = strong.text.split(":")
#for word in split_meta:
#    print(word.strip())
print("position is",split_meta[1].strip().split(" ")[0], "and weight is")
meta2 = split_meta[1].strip().split("\n")
print(meta2[0].strip())
print(meta2[4].strip())
meta3=meta2[4].strip().split(",")
print(meta3)



for child in results[0].children:
    print("-- children--")
    print(child)
print("-- strong--")
print(strong)
print(strong.text)
spans = results[0].find_all('<span>')
print(spans)

# Retrieve the parent divs for all articles
results = soup.find_all('div', class_='players')

# loop over results to get article data
for result in results:
    # scrape the article header 
    header = result.find('h1', class_='article-item__headline').text
    
    # scrape the article subheader
    subheader = result.find('h2', class_='article-item__subheader').text
    
    # scrape the datetime
    datetime = result.find('span', class_='article-item__date')['data-date'] 
    
    # get only the date from the datetime
    date = datetime.split('T')[0]
    
    # print article data
    print('-----------------')
    print(header)
    print(subheader)
    print(date)